In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('Level_2_ZyMoC1_EsCoC2_PsPuC3_1188.csv') 

# split data
X=data.iloc[:,51:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

      Schneider.Xr.A  Schneider.Xr.R  ...  Grantham.Xd.29  Grantham.Xd.30
0           0.084182        0.061733  ...        0.032505        0.032749
1           0.120118        0.083347  ...        0.031626        0.032946
2           0.129916        0.043305  ...        0.031596        0.031098
3           0.124517        0.045875  ...        0.031188        0.031233
4           0.144744        0.057897  ...        0.031939        0.030060
...              ...             ...  ...             ...             ...
1183        0.062749        0.038615  ...        0.029650        0.032397
1184        0.084790        0.064235  ...        0.032343        0.028946
1185        0.163940        0.032788  ...        0.033358        0.028789
1186        0.105041        0.010004  ...        0.032696        0.027279
1187        0.105041        0.010004  ...        0.032696        0.027279

[1188 rows x 100 columns]
0       1
1       1
2       1
3       1
4       1
       ..
1183    3
1184    3
1185 

In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.20,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(100, 3 - 1) = 2 components.
  ChangedBehaviorWarn

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(3 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 100)            40800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 128)            117248    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 207,651
Trainable params: 207,651
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
44/44 [==============================] - 31s 72ms/step - loss: 0.6768 - accuracy: 0.4286 - val_loss: 0.5606 - val_accuracy: 0.5304
Epoch 2/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.5643 - accuracy: 0.5004 - val_loss: 0.5144 - val_accuracy: 0.6087
Epoch 3/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.5474 - accuracy: 0.5796 - val_loss: 0.4511 - val_accuracy: 0.6841
Epoch 4/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.5022 - accuracy: 0.6042 - val_loss: 0.4143 - val_accuracy: 0.6928
Epoch 5/1000
44/44 [==============================] - 1s 14ms/step - loss: 0.4764 - accuracy: 0.6228 - val_loss: 0.4061 - val_accuracy: 0.7072
Epoch 6/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.4773 - accuracy: 0.6176 - val_loss: 0.4078 - val_accuracy: 0.7188
Epoch 7/1000
44/44 [==============================] - 1s 17ms/step - loss: 0.4606 - accuracy: 0.6410 - val_loss: 0.4090 - val_accuracy: 0.707

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.6623093681917211
test-acc = 0.6318840579710145
train-CFR =               precision    recall  f1-score   support

           0       0.69      0.66      0.67       453
           1       0.63      0.62      0.62       469
           2       0.67      0.71      0.69       455

    accuracy                           0.66      1377
   macro avg       0.66      0.66      0.66      1377
weighted avg       0.66      0.66      0.66      1377

test-CFR =               precision    recall  f1-score   support

           0       0.72      0.67      0.70       121
           1       0.57      0.61      0.59       105
           2       0.61      0.61      0.61       119

    accuracy                           0.63       345
   macro avg       0.63      0.63      0.63       345
weighted avg       0.64      0.63      0.63       345

